In [101]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from scipy.stats import entropy

df = pd.read_csv("../raw/amazon_sample.csv")
df


,Unnamed: 0,product_name,Average_rating,Num_of_Rating,Rating,review_title,Review_Text,Posted_Date,Helpfulness,has_images,price,Day_elapsed
0,0,Energizer Rechargeable AA and AAA Battery Char...,4.7,62436,5.0,look very good,This is an excellent battery charger.,2022-11-08,0,0,11.39,149
1,1,Fintie Rotating Case for iPad 6th / 5th Genera...,4.5,25873,3.0,Not the greatest quality,Fell apart at folding parts. Ink started comin...,2020-01-18,0,0,13.99,1182
2,2,Midland - WR120B - NOAA Emergency Weather Aler...,4.6,18908,5.0,This is the most comprehensive weather radio I...,This radio covers all types of natural weather...,2023-01-20,0,0,39.99,217
3,3,"Amazon Basics HDMI to DVI Adapter Cable, Bi-Di...",4.7,81186,1.0,typical,"broke after 4 months. rather, it simply stoppe...",2020-06-02,0,0,8.54,1026
4,4,"RCA to HDMI, GANA 1080P Mini RCA Composite CVB...",4.4,18061,5.0,Ease of Use,Connected up easily between our TV (with HDMI)...,2020-01-01,0,0,6.99,1169
...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,"Screen Cleaner Kit - Best for LED & LCD TV, Co...",4.7,38368,4.0,Muy bueno,Muy bueno,2021-05-08,0,0,19.99,683
996,996,Amazon Basics 3.5mm Aux Jack Audio Extension C...,4.6,57765,5.0,Works well,Bought this for my wife to take to work. Her ...,2020-03-27,0,0,6.99,1150
997,997,"Handsfree Call Car Charger,Wireless Bluetooth ...",4.3,45909,1.0,DRAINED CAR BATTERY,This is a great product however after having i...,2020-08-05,0,0,16.99,980
998,998,Sanus Adjustable Height Speaker Stand - Extend...,4.5,19804,5.0,Vary Sturdy Fit Perfectly,Fit my high end Logitech dolby front speakers ...,2020-02-09,0,0,45.99,1187


# length

In [104]:
def count_words(column):
    return column.apply(lambda x: len(str(x).split()))

# 각 열의 단어 수 계산하여 새로운 열 추가
df['title_length'] = count_words(df['review_title'])
df['text_length'] = count_words(df['Review_Text'])

# 결과 출력
df

,Unnamed: 0,product_name,Average_rating,Num_of_Rating,Rating,review_title,Review_Text,Posted_Date,Helpfulness,has_images,price,Day_elapsed,title_length,text_length
0,0,Energizer Rechargeable AA and AAA Battery Char...,4.7,62436,5.0,look very good,This is an excellent battery charger.,2022-11-08,0,0,11.39,149,3,6
1,1,Fintie Rotating Case for iPad 6th / 5th Genera...,4.5,25873,3.0,Not the greatest quality,Fell apart at folding parts. Ink started comin...,2020-01-18,0,0,13.99,1182,4,18
2,2,Midland - WR120B - NOAA Emergency Weather Aler...,4.6,18908,5.0,This is the most comprehensive weather radio I...,This radio covers all types of natural weather...,2023-01-20,0,0,39.99,217,10,74
3,3,"Amazon Basics HDMI to DVI Adapter Cable, Bi-Di...",4.7,81186,1.0,typical,"broke after 4 months. rather, it simply stoppe...",2020-06-02,0,0,8.54,1026,1,19
4,4,"RCA to HDMI, GANA 1080P Mini RCA Composite CVB...",4.4,18061,5.0,Ease of Use,Connected up easily between our TV (with HDMI)...,2020-01-01,0,0,6.99,1169,3,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,"Screen Cleaner Kit - Best for LED & LCD TV, Co...",4.7,38368,4.0,Muy bueno,Muy bueno,2021-05-08,0,0,19.99,683,2,2
996,996,Amazon Basics 3.5mm Aux Jack Audio Extension C...,4.6,57765,5.0,Works well,Bought this for my wife to take to work. Her ...,2020-03-27,0,0,6.99,1150,2,39
997,997,"Handsfree Call Car Charger,Wireless Bluetooth ...",4.3,45909,1.0,DRAINED CAR BATTERY,This is a great product however after having i...,2020-08-05,0,0,16.99,980,3,28
998,998,Sanus Adjustable Height Speaker Stand - Extend...,4.5,19804,5.0,Vary Sturdy Fit Perfectly,Fit my high end Logitech dolby front speakers ...,2020-02-09,0,0,45.99,1187,4,9


# time_lapesd

In [107]:
# Posted_Date와 Crawled_date를 datetime 형식으로 변환
df['Posted_Date'] = pd.to_datetime(df['Posted_Date'], format='%Y-%m-%d', errors='coerce')
df['Crawled_date'] = pd.to_datetime(df['Crawled_date'], format='%Y-%m-%d', errors='coerce')

# 날짜 차이 계산 (Crawled_date - Posted_Date)
df['time_lapsed'] = (df['Crawled_date'] - df['Posted_Date']).dt.days  # 일(day) 단위로 계산

df

KeyError: 'Crawled_date'

# star_dev

In [110]:
df['Deviation of star ratings'] = abs(df['Rating'] - df['Average_Rating'])
df

,Unnamed: 0,product_name,Average_rating,Num_of_Rating,Rating,review_title,Review_Text,Posted_Date,Helpfulness,has_images,price,Day_elapsed,title_length,text_length,Deviation of star ratings
0,0,Energizer Rechargeable AA and AAA Battery Char...,4.7,62436,5.0,look very good,This is an excellent battery charger.,2022-11-08,0,0,11.39,149,3,6,0.3
1,1,Fintie Rotating Case for iPad 6th / 5th Genera...,4.5,25873,3.0,Not the greatest quality,Fell apart at folding parts. Ink started comin...,2020-01-18,0,0,13.99,1182,4,18,1.5
2,2,Midland - WR120B - NOAA Emergency Weather Aler...,4.6,18908,5.0,This is the most comprehensive weather radio I...,This radio covers all types of natural weather...,2023-01-20,0,0,39.99,217,10,74,0.4
3,3,"Amazon Basics HDMI to DVI Adapter Cable, Bi-Di...",4.7,81186,1.0,typical,"broke after 4 months. rather, it simply stoppe...",2020-06-02,0,0,8.54,1026,1,19,3.7
4,4,"RCA to HDMI, GANA 1080P Mini RCA Composite CVB...",4.4,18061,5.0,Ease of Use,Connected up easily between our TV (with HDMI)...,2020-01-01,0,0,6.99,1169,3,53,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,"Screen Cleaner Kit - Best for LED & LCD TV, Co...",4.7,38368,4.0,Muy bueno,Muy bueno,2021-05-08,0,0,19.99,683,2,2,0.7
996,996,Amazon Basics 3.5mm Aux Jack Audio Extension C...,4.6,57765,5.0,Works well,Bought this for my wife to take to work. Her ...,2020-03-27,0,0,6.99,1150,2,39,0.4
997,997,"Handsfree Call Car Charger,Wireless Bluetooth ...",4.3,45909,1.0,DRAINED CAR BATTERY,This is a great product however after having i...,2020-08-05,0,0,16.99,980,3,28,3.3
998,998,Sanus Adjustable Height Speaker Stand - Extend...,4.5,19804,5.0,Vary Sturdy Fit Perfectly,Fit my high end Logitech dolby front speakers ...,2020-02-09,0,0,45.99,1187,4,9,0.5


# readability

In [112]:
import textstat


# 4. Readability 점수를 계산할 열 이름 설정
text_column = "Review_Text"  # CSV 파일의 텍스트가 저장된 열 이름

# 5. 새로운 열 추가 및 점수 계산
df["FOG Index"] = df[text_column].apply(lambda text: textstat.gunning_fog(str(text)))
df["Flesch Reading Ease"] = df[text_column].apply(lambda text: textstat.flesch_reading_ease(str(text)))

df

,Unnamed: 0,product_name,Average_rating,Num_of_Rating,Rating,review_title,Review_Text,Posted_Date,Helpfulness,has_images,price,Day_elapsed,title_length,text_length,Deviation of star ratings,FOG Index,Flesch Reading Ease
0,0,Energizer Rechargeable AA and AAA Battery Char...,4.7,62436,5.0,look very good,This is an excellent battery charger.,2022-11-08,0,0,11.39,149,3,6,0.3,9.07,56.93
1,1,Fintie Rotating Case for iPad 6th / 5th Genera...,4.5,25873,3.0,Not the greatest quality,Fell apart at folding parts. Ink started comin...,2020-01-18,0,0,13.99,1182,4,18,1.5,3.60,96.18
2,2,Midland - WR120B - NOAA Emergency Weather Aler...,4.6,18908,5.0,This is the most comprehensive weather radio I...,This radio covers all types of natural weather...,2023-01-20,0,0,39.99,217,10,74,0.4,18.58,42.38
3,3,"Amazon Basics HDMI to DVI Adapter Cable, Bi-Di...",4.7,81186,1.0,typical,"broke after 4 months. rather, it simply stoppe...",2020-06-02,0,0,8.54,1026,1,19,3.7,3.80,78.75
4,4,"RCA to HDMI, GANA 1080P Mini RCA Composite CVB...",4.4,18061,5.0,Ease of Use,Connected up easily between our TV (with HDMI)...,2020-01-01,0,0,6.99,1169,3,53,0.6,8.34,74.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,"Screen Cleaner Kit - Best for LED & LCD TV, Co...",4.7,38368,4.0,Muy bueno,Muy bueno,2021-05-08,0,0,19.99,683,2,2,0.7,0.80,120.21
996,996,Amazon Basics 3.5mm Aux Jack Audio Extension C...,4.6,57765,5.0,Works well,Bought this for my wife to take to work. Her ...,2020-03-27,0,0,6.99,1150,2,39,0.4,7.25,83.66
997,997,"Handsfree Call Car Charger,Wireless Bluetooth ...",4.3,45909,1.0,DRAINED CAR BATTERY,This is a great product however after having i...,2020-08-05,0,0,16.99,980,3,28,3.3,8.46,57.27
998,998,Sanus Adjustable Height Speaker Stand - Extend...,4.5,19804,5.0,Vary Sturdy Fit Perfectly,Fit my high end Logitech dolby front speakers ...,2020-02-09,0,0,45.99,1187,4,9,0.5,8.04,62.34


# 파일 저장

In [114]:
df.to_csv("../temp/amazon_sampled_temp.csv")